In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [6]:
INTERACTION_MULTIPLIER = 3
original_data = pd.read_csv('dissecting-bias-master-data/data/data_new.csv').rename(columns={'gagne_sum_t':'Chronic_illnesses'})
original_data.columns = [a.capitalize().replace('-', '_') for a in original_data.columns]
original_data['Sex'] = original_data['Dem_female'].map(lambda x:'Female' if x == 1 else 'Male')
original_data['Age'] = 'Over 65'
original_data.loc[(original_data['Dem_age_band_55_64_tm1'] == 1) | (original_data['Dem_age_band_45_54_tm1'] == 1), 'Age'] = '45-64'
original_data.loc[(original_data['Dem_age_band_18_24_tm1'] == 1) | 
                  (original_data['Dem_age_band_25_34_tm1'] == 1) | 
                (original_data['Dem_age_band_35_44_tm1'] == 1), 'Age'] = '18-44'

# do some analysis
df = []
for column in original_data.columns:
    if column in ['Race', 'Sex', 'Age', 'Chronic_illnesses']:
        continue
    if 'Dem_age_band' in column:
        continue
    if not column.endswith('_tm1'):
        continue
    if original_data[column].std() < 1e-6:
        continue
    if original_data[column].mean() < 0.01:
        print("%s has mean %2.3f; skipping" % (column, original_data[column].mean()))
        continue
    original_data['z_score_x'] = (original_data[column] - original_data[column].mean())/original_data[column].std()
    
    base_model = sm.OLS.from_formula('Chronic_illnesses ~ z_score_x', data=original_data).fit()
    interaction_model = sm.OLS.from_formula('Chronic_illnesses ~ z_score_x*Sex', data=original_data).fit()
    
    df.append({"c":column, 
               "absolute_value_base_coef":abs(base_model.params['z_score_x']), 
               "base_p":base_model.pvalues['z_score_x'], 
               "base_R^2":base_model.rsquared,
               'boost_in_R^2_from_interaction':interaction_model.rsquared - base_model.rsquared})
    #"absolute_value_interaction_coef":abs(interaction_model.params['z_score_x:Sex[T.Male]']), 
    # "interaction_p":interaction_model.pvalues['z_score_x:Sex[T.Male]']})
df = pd.DataFrame(df)

cols_to_keep = []
# take columns with large interactions with sex. 
#df = df.sort_values(by='absolute_value_base_coef')[::-1]
#cols_to_keep = list(df['c'].iloc[:5])
df = df.sort_values(by='boost_in_R^2_from_interaction')[::-1]
cols_to_keep = cols_to_keep + list(df['c'].iloc[:10])


# THIS ADDS SYNTHETIC INTERACTION
base_model = sm.OLS.from_formula('Chronic_illnesses ~ %s' % '+'.join(cols_to_keep), data=original_data).fit()
interaction_effect = INTERACTION_MULTIPLIER * np.dot(original_data[base_model.params.index[1:]].values, base_model.params[1:])
data_to_make_synthetic_idxs = original_data['Sex'] == 'Male'
original_data.loc[data_to_make_synthetic_idxs, 'Chronic_illnesses'] += interaction_effect[data_to_make_synthetic_idxs]
original_data['Chronic_illnesses'] = original_data['Chronic_illnesses'].map(lambda x:max(int(x), 0))
print("Data on chronic illnesses")
print(original_data['Chronic_illnesses'].describe())

cols_to_keep = ['Chronic_illnesses','Race','Sex','Age'] + cols_to_keep
original_data = original_data[cols_to_keep]



original_data = original_data.sample(frac=1, random_state=42)

train_data = original_data.iloc[:40000]
test_data = original_data.iloc[40000:]
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)


Alcohol_elixhauser_tm1 has mean 0.009; skipping
Bloodlossanemia_elixhauser_tm1 has mean 0.002; skipping
Drugabuse_elixhauser_tm1 has mean 0.006; skipping
Paralysis_elixhauser_tm1 has mean 0.001; skipping
Pulmcirc_elixhauser_tm1 has mean 0.006; skipping
Wtloss_elixhauser_tm1 has mean 0.001; skipping
Dementia_romano_tm1 has mean 0.009; skipping
Hemiplegia_romano_tm1 has mean 0.003; skipping
Hivaids_romano_tm1 has mean 0.003; skipping
Metastatic_romano_tm1 has mean 0.006; skipping
Ulcer_romano_tm1 has mean 0.005; skipping
Crp_tests_tm1 has mean 0.000; skipping
Crp_min_low_tm1 has mean 0.000; skipping
Crp_min_high_tm1 has mean 0.000; skipping
Crp_min_normal_tm1 has mean 0.000; skipping
Crp_mean_low_tm1 has mean 0.000; skipping
Crp_mean_high_tm1 has mean 0.000; skipping
Crp_mean_normal_tm1 has mean 0.000; skipping
Crp_max_low_tm1 has mean 0.000; skipping
Crp_max_high_tm1 has mean 0.000; skipping
Crp_max_normal_tm1 has mean 0.000; skipping
Ghba1c_min_low_tm1 has mean 0.000; skipping
Ghba1c_m

In [ ]:
interaction_effect

In [69]:
for formula in ['Chronic_illnesses ~ Age * Cost_other_tm1', 
                'Chronic_illnesses ~ Cost_other_tm1']:
    print(formula)
    model = sm.OLS.from_formula(formula, data=train_data.iloc[:11100]).fit()
    test_preds = model.predict(test_data)
    rmse = np.sqrt(np.mean((test_preds - test_data['Chronic_illnesses']) ** 2))
    print(model.summary())
    print(rmse)

Chronic_illnesses ~ Age * Cost_other_tm1
                            OLS Regression Results                            
Dep. Variable:      Chronic_illnesses   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     407.4
Date:                Fri, 20 May 2022   Prob (F-statistic):               0.00
Time:                        14:41:27   Log-Likelihood:                -22193.
No. Observations:               11100   AIC:                         4.440e+04
Df Residuals:                   11094   BIC:                         4.444e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [66]:
df.sort_values(by='boost_in_R^2_from_interaction')[::-1]

,c,absolute_value_base_coef,base_p,base_R^2,boost_in_R^2_from_interaction
42,Cost_other_tm1,0.375990,0.000000e+00,0.037466,0.111024
39,Cost_op_primary_care_tm1,0.154918,1.159428e-69,0.006360,0.108629
40,Cost_op_specialists_tm1,0.371423,0.000000e+00,0.036561,0.107745
65,Cre_max_low_tm1,0.033014,1.739849e-04,0.000289,0.106624
102,Ldl_min_high_tm1,0.032179,2.531283e-04,0.000274,0.105977
...,...,...,...,...,...
57,Sodium_tests_tm1,0.860410,0.000000e+00,0.196198,0.071807
20,Uncompdiabetes_elixhauser_tm1,0.878116,0.000000e+00,0.204356,0.064116
2,Arrhythmia_elixhauser_tm1,0.801854,0.000000e+00,0.170402,0.061080
10,Hypertension_elixhauser_tm1,1.008239,0.000000e+00,0.269408,0.017674


In [50]:
train_data.iloc[0]

Chronic_illnesses                     2
Race                              white
Sex                              Female
Age                               45-64
Gagne_sum_tm1                         2
Hypertension_elixhauser_tm1           1
Uncompdiabetes_elixhauser_tm1         0
Cre_tests_tm1                         0
Sodium_tests_tm1                      0
Cre_max_normal_tm1                    0
Cre_min_low_tm1                       0
Cost_other_tm1                    400.0
Cre_max_high_tm1                      0
Cre_min_high_tm1                      0
Cre_mean_high_tm1                     0
Hct_min_low_tm1                       0
Cre_mean_normal_tm1                   0
Hct_mean_low_tm1                      0
Cre_mean_low_tm1                      0
Name: 9224, dtype: object

In [71]:
train_data['Crp_mean_normal_tm1'].mean()

2.5e-05